In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import json

In [2]:
# hours = []
#Get first 24 hours worth of data on day 1
# for i in range(24): 
#     hours.append(pd.read_json("./data/combined1_" + str(i), lines=True))
# hour0 = pd.read_json("./combined1_0", lines=True)
# hour1 = pd.read_json("./combined1_1", lines=True)
# hour19 = pd.read_json("./combined1_19", lines=True)

In [3]:
# table = pd.concat(hours)

In [4]:
# table.to_pickle("Day1")
table = pd.read_pickle("./data/Day1")

In [5]:
# Read in file chunksize at a time to prevent kernel from dying, read_json will return an iterator
# table = pd.read_json("./data/combined_all_days", chunksize=2500000, lines=True)
# table = pd.concat(table)

## Data Exploration

In [6]:
table.shape

(4783760, 42)

In [7]:
table.head()

,ad_network_id,ad_type,advertiser_id,bid_requests,bid_responses,c_cnt,c_timestamp,c_txn_fee,c_txn_rate,campaign_id,...,txn_fee,txn_rate,ua_device,ua_device_type,ua_name,ua_os_name,vi_cnt,vi_timestamp,vv_cnt,zone_id
0,1839,[story],4523,[],[],0.0,NaN,NaN,NaN,26027,...,NaN,NaN,Samsung SM-G930V,MOB,Chrome Mobile,Android,0.0,NaN,0.0,26024
1,977,[story],2917,[],[],NaN,NaN,NaN,NaN,25990,...,NaN,NaN,iPad,TAB,Chrome Mobile iOS,iOS,NaN,NaN,NaN,26024
2,1828,[story],4514,[],[],0.0,NaN,NaN,NaN,22360,...,NaN,NaN,QTAIR7,TAB,Chrome,Android,0.0,NaN,0.0,17681
3,1839,[story],4523,[],[],0.0,NaN,NaN,NaN,26027,...,NaN,NaN,iPhone,MOB,Mobile Safari,iOS,0.0,NaN,0.0,26024
4,727,[story],5152,[],[],NaN,NaN,NaN,NaN,26038,...,NaN,NaN,iPad,TAB,Mobile Safari UI/WKWebView,iOS,NaN,NaN,NaN,14414


In [8]:
filtered = table[(~np.isnan(table["c_cnt"]))]
filtered.dropna(subset=["i_timestamp"], inplace=True)
filtered.shape

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


(3799091, 42)

In [9]:
size = filtered.shape[0]
size

3799091

In [10]:
filtered.c_cnt.value_counts()

0.0    3796692
1.0       2399
Name: c_cnt, dtype: int64

In [11]:
CLICKED_CNT = filtered.c_cnt.value_counts()[1]
CLICKED_CNT

2399

In [12]:
# advertiser_click_rate = (filtered[["advertiser_id", "c_cnt"]].groupby("advertiser_id").sum() 
#  / filtered[["advertiser_id", "c_cnt"]].groupby("advertiser_id").count()).reset_index()
# filtered = pd.merge(filtered, advertiser_click_rate, on="advertiser_id")
# filtered.head()

In [13]:
#Find average ctr rate of each advertiser
def score_advertisers(table, score_func):
    click_counts = table[["advertiser_id", "c_cnt"]]
    clean_ccnt = click_counts.dropna()
    grouped = clean_ccnt.groupby("advertiser_id")
#     merged = pd.merge(clean_ccnt, ADVERTISERS)
#     grouped = merged.groupby("advertiser_name")
    score_means = lambda x: score_func(np.mean(x))
    agg_scored = grouped.agg({'c_cnt': score_means})
#     agg_scored = agg_scored.sort_values(by="c_cnt", ascending=False)
    return agg_scored

click_rates = score_advertisers(filtered, lambda x: x).reset_index()

In [14]:
click_rates.rename(columns={"c_cnt": "ctr_rate"}, inplace=True)
filtered = pd.merge(filtered, click_rates, on="advertiser_id")
filtered.head()

,ad_network_id,ad_type,advertiser_id,bid_requests,bid_responses,c_cnt,c_timestamp,c_txn_fee,c_txn_rate,campaign_id,...,txn_rate,ua_device,ua_device_type,ua_name,ua_os_name,vi_cnt,vi_timestamp,vv_cnt,zone_id,ctr_rate
0,1839,[story],4523,[],[],0.0,NaN,NaN,NaN,26027,...,NaN,Samsung SM-G930V,MOB,Chrome Mobile,Android,0.0,NaN,0.0,26024,0.0001
1,1839,[story],4523,[],[],0.0,NaN,NaN,NaN,26027,...,NaN,iPhone,MOB,Mobile Safari,iOS,0.0,NaN,0.0,26024,0.0001
2,1839,[story],4523,[],[],0.0,NaN,NaN,NaN,26027,...,NaN,Other,PC,Chrome,Windows 8.1,0.0,NaN,0.0,26024,0.0001
3,1839,[story],4523,[],[],0.0,NaN,NaN,NaN,26027,...,NaN,iPhone,MOB,Mobile Safari,iOS,0.0,NaN,0.0,26024,0.0001
4,1839,[story],4523,[],[],0.0,NaN,NaN,NaN,26027,...,NaN,Other,PC,Chrome,Windows 7,0.0,NaN,0.0,26024,0.0001


In [15]:
# filtered.groupby(["advertiser_id"])["c_cnt"].sum() / filtered.groupby(["advertiser_id"])["vi_cnt"].sum()

In [16]:
filtered["ua_device_type"].value_counts()

MOB    1935685
PC     1233963
TAB     612025
BOT       3118
Name: ua_device_type, dtype: int64

In [17]:
# sns.distplot(filtered["r_num_ads_requested"])
# plt.show()

In [18]:
filtered.campaign_type.value_counts()

private    3623244
outside     175847
Name: campaign_type, dtype: int64

In [19]:
filtered.columns

Index(['ad_network_id', 'ad_type', 'advertiser_id', 'bid_requests',
       'bid_responses', 'c_cnt', 'c_timestamp', 'c_txn_fee', 'c_txn_rate',
       'campaign_id', 'campaign_type', 'cr_cnt', 'creative_id', 'exp_mode',
       'f_cnt', 'geo_continent_code', 'geo_country_code2', 'geo_dma_code',
       'geo_region_name', 'geo_timezone', 'i_cnt', 'i_timestamp',
       'pub_network_id', 'r_cnt', 'r_num_ads_requested', 'r_num_ads_returned',
       'r_num_ads_third_party', 'r_timestamp', 'rate_metric', 'session_id',
       'site_id', 'token', 'txn_fee', 'txn_rate', 'ua_device',
       'ua_device_type', 'ua_name', 'ua_os_name', 'vi_cnt', 'vi_timestamp',
       'vv_cnt', 'zone_id', 'ctr_rate'],
      dtype='object')

In [20]:
filtered.ua_os_name.value_counts()

iOS                     1586539
Android                  953202
Windows 10               604754
Windows 7                258308
Mac OS X                 255352
Windows 8.1               67141
Chrome OS                 13584
Windows Vista             13100
Linux                     11853
Windows XP                 9755
Windows 8                  8077
Ubuntu                     6123
Other                      3113
BlackBerry OS              2755
Windows Phone              2589
Windows RT 8.1             2027
Fedora                      464
Tizen                       112
Windows RT                   49
BlackBerry Tablet OS         35
GoogleTV                     21
Windows 2000                 21
FreeBSD                      19
Kindle                       19
Symbian OS                   15
webOS                        12
OpenBSD                      11
Windows NT 4.0               10
Windows                       9
Firefox OS                    8
Linux Mint                    5
Windows 

## Feature Engineering and Model Creation

In [21]:
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [22]:
# Shuffle data
filtered = shuffle(filtered)

In [23]:
def generate_sample(n):
    """Return X matrix with n no click samples (undersampling)"""
    clicked = filtered[filtered["c_cnt"] == 1]
    no_click = filtered[filtered["c_cnt"] == 0].sample(n)
    return shuffle(pd.concat([clicked, no_click]))

def generate_subset(n):
    """Returns random subset of original matrix with n/2 click samples and n/2 nonclicks"""
    clicked = filtered[filtered["c_cnt"] == 1].reset_index(drop=True)
    no_click = filtered[filtered["c_cnt"] == 0].reset_index(drop=True)
    click_idx = np.random.randint(n, size=int(n/2))
    no_click_idx = np.random.randint(no_click.shape[0], size=int(n/2))
    return (clicked.drop(click_idx), no_click.drop(no_click_idx), 
            shuffle(pd.concat([clicked.iloc[click_idx, :], no_click.iloc[no_click_idx, :]])))

In [31]:
# Create X and Y matrix for model
click_filtered, no_click_filtered, sample = generate_subset(CLICKED_CNT)
# sample = generate_sample(CLICKED_CNT)
sample.dropna(subset=["ua_device_type"], inplace=True)

In [32]:
X_data = pd.DataFrame()
y_data = sample["c_cnt"]
y_data.shape

(2391,)

In [33]:
sample["ua_device_type"].value_counts()

MOB    1032
PC      709
TAB     648
BOT       2
Name: ua_device_type, dtype: int64

In [34]:
# One hot encode device type feature
device_types = sample["ua_device_type"].tolist()
le_device = preprocessing.LabelBinarizer()
transformed_device = le_device.fit_transform(device_types)
X_data["bot"] = transformed_device[:, 0]
X_data["mobile"] = transformed_device[:, 1]
X_data["pc"] = transformed_device[:, 2]
X_data["tablet"] = transformed_device[:, 3]

In [35]:
def transform_hour(x):
    """Assigns hour to a bin"""
    if x >= 5 and x < 8: return 1 #early morning 
    elif x >= 8 and x < 11: return 2 #morning 
    elif x >= 11 and x < 14: return 3 #midday
    elif x >= 14 and x < 19: return 4 #afternoon
    elif x >= 19 and x < 22: return 5 #evening
    else: return 6 #night

In [36]:
def create_numerical_features(df, sample):
    """Add numerical features to dataframe df from sample df"""
    df["i_cnt"] = sample["i_cnt"].tolist()
    df["r_cnt"] = sample["r_cnt"].tolist()
    df["campaign_type"] = sample["campaign_type"].apply(lambda x: 1 if x == "private" else 0).tolist()
    df["ctr_rate"] = sample["ctr_rate"].tolist()
    df["num_ads_requested"] = sample["r_num_ads_requested"].tolist()
    # Create hour feature
    hours = sample["i_timestamp"].str[11:13] # get hour of timestamp column
    # X_data["hour"] = np.sin(hours.astype(int)).tolist() #sine transform hour because it's cyclical
    df["hour"] = hours.astype(int).apply(transform_hour).tolist()

In [37]:
create_numerical_features(X_data, sample)

In [38]:
# Hash sparse features, allows for less sparse representation than one hot encoding
# More info can be found on https://en.wikipedia.org/wiki/Feature_hashing
from sklearn.feature_extraction import FeatureHasher
h = FeatureHasher(n_features=30, input_type='string')

hashed_ad_id = h.transform(sample["advertiser_id"].astype(str)).toarray()
hashed_adnet_id = h.transform(sample["ad_network_id"].astype(str)).toarray()
hashed_country_code =  h.transform(sample["geo_country_code2"]).toarray()
hashed_os = h.transform(sample["ua_os_name"]).toarray()
hashed_site_id = h.transform(sample["site_id"].astype(str)).toarray()
hashed_site_id

array([[ 1.,  0., -1., ...,  0.,  0.,  0.],
       [ 1.,  0., -1., ...,  0.,  0.,  0.],
       [ 0.,  0., -1., ...,  0.,  0.,  0.],
       ..., 
       [ 1.,  0., -1., ...,  0.,  0.,  0.],
       [ 0.,  0., -1., ...,  0.,  0.,  0.],
       [ 0.,  0., -1., ...,  0.,  0.,  0.]])

In [39]:
# One hot encode continent code 
le_cont = preprocessing.LabelBinarizer()
transformed_continent = le_cont.fit_transform(sample["geo_continent_code"])

# fitted_advertisers = le.fit(top_8_advertisers)
# transformed_ad_id = fitted_advertisers.transform(sample["advertiser_id"])

X_data = pd.concat([X_data, pd.DataFrame(transformed_continent), pd.DataFrame(hashed_ad_id),
                            pd.DataFrame(hashed_country_code), pd.DataFrame(hashed_adnet_id),
                            pd.DataFrame(hashed_os), pd.DataFrame(hashed_site_id)], 
                   axis=1)

In [40]:
X_data.shape

(2391, 166)

In [41]:
X_data.head()

,bot,mobile,pc,tablet,i_cnt,r_cnt,campaign_type,ctr_rate,num_ads_requested,hour,...,20,21,22,23,24,25,26,27,28,29
0,0,0,0,1,1.0,1,1,0.001161,10,6,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,1,0,1.0,1,1,0.000100,10,4,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,1,0,1.0,1,0,0.007853,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1,0,0,1.0,1,1,0.000100,4,6,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,1,0,0,1.0,1,1,0.000100,10,4,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Bagging

In [42]:
bg = ensemble.BaggingClassifier()
# bg.fit(X_train, y_train)
# bg.score(X_test, y_test) #test accuracy
bg_scores = cross_val_score(bg, X_data, y_data, cv=5)
print("Bagging Classifier Avg Score:", np.mean(bg_scores))

Bagging Classifier Avg Score: 0.773756854767


### Boosting

In [43]:
# ADABOOST
adaboost = ensemble.AdaBoostClassifier()
ada_scores = cross_val_score(adaboost, X_data, y_data, cv=5)
print("Boosting Classifier Avg Score:", np.mean(ada_scores))

Boosting Classifier Avg Score: 0.787549554834


In [44]:
# XGBOOST
xgboost = ensemble.GradientBoostingClassifier()
xg_scores = cross_val_score(xgboost, X_data, y_data, cv=5)
print("Boosting Classifier Avg Score:", np.mean(xg_scores))

Boosting Classifier Avg Score: 0.795502002777


In [45]:
#Tune XGBoost parameters
parameters = {'max_depth':[3, 5, 7]}
clf = GridSearchCV(xgboost, parameters, cv=5)
clf.fit(X_data, y_data)
print("Optimal Tree Depth:", clf.best_params_)

Optimal Tree Depth: {'max_depth': 3}


## Testing on original data

In [46]:
#Only keep rows that weren't part of training set
filtered_subset = shuffle(pd.concat([click_filtered, no_click_filtered]))

In [47]:
filtered_subset.shape

(3796941, 43)

In [48]:
X_test = pd.DataFrame()
y_test = filtered_subset.c_cnt

In [49]:
create_numerical_features(X_test, filtered_subset)

In [50]:
test_transformed = le_device.transform(filtered_subset['ua_device_type'].tolist())
X_test["bot"] = test_transformed[:, 0]
X_test["mobile"] = test_transformed[:, 1]
X_test["pc"] = test_transformed[:, 2]
X_test["tablet"] = test_transformed[:, 3]

In [51]:
hashed_ad_id_test = h.transform(filtered_subset["advertiser_id"].astype(str)).toarray()
hashed_adnet_id_test = h.transform(filtered_subset["ad_network_id"].astype(str)).toarray()
hashed_country_code_test =  h.transform(filtered_subset["geo_country_code2"]).toarray()
hashed_os_test = h.transform(filtered_subset["ua_os_name"]).toarray()
hashed_site_id_test = h.transform(filtered_subset["site_id"].astype(str)).toarray()

In [52]:
# One hot encode continent code
test_transformed_continent = le_cont.transform(filtered_subset["geo_continent_code"])

X_test = pd.concat([X_test, pd.DataFrame(test_transformed_continent), pd.DataFrame(hashed_ad_id_test),
                            pd.DataFrame(hashed_country_code_test), pd.DataFrame(hashed_adnet_id_test),
                            pd.DataFrame(hashed_os_test), pd.DataFrame(hashed_site_id_test)], 
                   axis=1)

In [53]:
X_test.head()

,i_cnt,r_cnt,campaign_type,ctr_rate,num_ads_requested,hour,bot,mobile,pc,tablet,...,20,21,22,23,24,25,26,27,28,29
0,1.0,1,1,0.001161,10,4,0,0,0,1,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1,1,0.000393,10,4,0,1,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1,1,0.000441,6,6,0,0,0,1,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1,1,0.000320,4,6,0,1,0,0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1,1,0.000100,10,4,0,1,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
X_test.shape

(3796941, 166)

In [55]:
xgboost.fit(X_data, y_data)
xgboost.score(X_test, y_test)

0.45484773137112217

In [56]:
y_pred = xgboost.predict(X_test)

In [57]:
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_test, y_pred) #00 - TN, 10 - FN, 01 - FP, 11 - TP
confusion

array([[1726459, 2069034],
       [    877,     571]])

Our precision - TP/(TP + FP) - is very low, which means we are predicting a lot of non-clicks as clicks:

In [58]:
precision = confusion[1][1] / (confusion[1][1] + confusion[0][1])
recall = confusion[1][1] / (confusion[1][1] + confusion[1][0])
precision, recall

(0.00027589805784195535, 0.39433701657458564)

As a result, our F1 score isn't optimal either:

In [59]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred)

0.00055141032122306866

# FFM (inspired by https://www.csie.ntu.edu.tw/~cjlin/papers/ffm.pdf)

In [60]:
import xlearn as xl
xl.hello()

We need to make a new dataframe with categorical features one-hot encoded instead of hashed:

In [61]:
sample.head()

,ad_network_id,ad_type,advertiser_id,bid_requests,bid_responses,c_cnt,c_timestamp,c_txn_fee,c_txn_rate,campaign_id,...,txn_rate,ua_device,ua_device_type,ua_name,ua_os_name,vi_cnt,vi_timestamp,vv_cnt,zone_id,ctr_rate
1731868,977,[story],2917,[],[],0.0,NaN,NaN,NaN,26010,...,NaN,iPad,TAB,Mobile Safari,iOS,0.0,NaN,0.0,26024,0.001161
2651542,1839,[story],4523,[],[],0.0,NaN,NaN,NaN,26027,...,NaN,Other,PC,Chrome,Windows 8.1,0.0,NaN,0.0,26024,0.000100
1850,727,[story],5152,[],[],1.0,2018-02-11T10:21:13.808000Z,0.0,0.5,26038,...,0.5,Other,PC,Firefox,Windows 10,1.0,2018-02-11T10:20:22.602000Z,0.0,14411,0.007853
1019508,1839,[story],4523,[],[],0.0,NaN,NaN,NaN,26027,...,NaN,Samsung SM-G900V,MOB,Chrome Mobile,Android,1.0,2018-02-11T01:36:59.467000Z,0.0,17681,0.000100
3086467,1839,[story],4523,[],[],0.0,NaN,NaN,NaN,26027,...,NaN,iPhone,MOB,Mobile Safari,iOS,0.0,NaN,0.0,26024,0.000100


In [62]:
sample_size = sample.shape[0]

In [63]:
# Split to help generate data for train and test ffm matrices
train_ffm_original = sample.iloc[:int(0.8*sample_size), :]
test_ffm_original = sample.iloc[int(0.8*sample_size):, :]

In [64]:
# These are the actual data that will be trained with and tested on
train_ffm_transformed = pd.DataFrame(data={'c_cnt':train_ffm_original["c_cnt"].tolist()})
test_ffm_transformed = pd.DataFrame(data={'c_cnt':test_ffm_original["c_cnt"].tolist()})

In [65]:
# Create numerical features
create_numerical_features(train_ffm_transformed, train_ffm_original)
create_numerical_features(test_ffm_transformed, test_ffm_original)

In [66]:
# Create encoders and one hot encode features
max_categories = 30

def encode_feature(feature, sample):
    """Encode given feature from sample df and return"""
    encoder = preprocessing.LabelBinarizer()
    encoder.fit(filtered[feature].value_counts()[:30]
                         .index
                         .tolist()
                )
    return encoder.transform(sample[feature])

ffm_train_encoded_device = le_device.transform(train_ffm_original["ua_device_type"])
ffm_train_encoded_ad_id = encode_feature("advertiser_id", train_ffm_original)
ffm_train_encoded_ad_network_id = encode_feature("ad_network_id", train_ffm_original)
ffm_train_encoded_geo_country_code = encode_feature("geo_country_code2", train_ffm_original)
ffm_train_encoded_ua_os_name = encode_feature("ua_os_name", train_ffm_original)
ffm_train_encoded_site_id = encode_feature("site_id", train_ffm_original)

ffm_test_encoded_device = le_device.transform(test_ffm_original["ua_device_type"])
ffm_test_encoded_ad_id = encode_feature("advertiser_id", test_ffm_original)
ffm_test_encoded_ad_network_id = encode_feature("ad_network_id", test_ffm_original)
ffm_test_encoded_geo_country_code = encode_feature("geo_country_code2", test_ffm_original)
ffm_test_encoded_ua_os_name = encode_feature("ua_os_name", test_ffm_original)
ffm_test_encoded_site_id = encode_feature("site_id", test_ffm_original)

In [67]:
feature_count = [] # Tracks number of columns per categorical feature, i.e. device_type has 4 columns
feature_count.append(ffm_train_encoded_device.shape[1])
feature_count.append(ffm_train_encoded_ad_id.shape[1])
feature_count.append(ffm_train_encoded_ad_network_id.shape[1])
feature_count.append(ffm_train_encoded_geo_country_code.shape[1])
feature_count.append(ffm_train_encoded_ua_os_name.shape[1])
feature_count.append(ffm_train_encoded_site_id.shape[1])
feature_count

[4, 26, 21, 30, 30, 24]

In [68]:
# Combined all encoded features
train_ffm_transformed = pd.concat([train_ffm_transformed, pd.DataFrame(ffm_train_encoded_device),
                                  pd.DataFrame(ffm_train_encoded_ad_id),
                                  pd.DataFrame(ffm_train_encoded_ad_network_id),
                                  pd.DataFrame(ffm_train_encoded_geo_country_code),
                                  pd.DataFrame(ffm_train_encoded_ua_os_name),
                                  pd.DataFrame(ffm_train_encoded_site_id)], axis=1)

In [69]:
test_ffm_transformed = pd.concat([test_ffm_transformed, pd.DataFrame(ffm_test_encoded_device),
                                  pd.DataFrame(ffm_test_encoded_ad_id),
                                  pd.DataFrame(ffm_test_encoded_ad_network_id),
                                  pd.DataFrame(ffm_test_encoded_geo_country_code),
                                  pd.DataFrame(ffm_test_encoded_ua_os_name),
                                  pd.DataFrame(ffm_test_encoded_site_id)], axis=1)

In [70]:
train_ffm_transformed.head()

,c_cnt,i_cnt,r_cnt,campaign_type,ctr_rate,num_ads_requested,hour,0,1,2,...,14,15,16,17,18,19,20,21,22,23
0,0.0,1.0,1,1,0.001161,10,6,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.0,1.0,1,1,0.000100,10,4,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1.0,1.0,1,0,0.007853,1,2,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0.0,1.0,1,1,0.000100,4,6,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,1.0,1,1,0.000100,10,4,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [71]:
# Based on Kaggle kernel by Scirpus
def convert_to_ffm(df,data_type,numerics):
    nrows = df.shape[0]
    ncols = df.shape[1]
    with open(str(data_type) + "_ffm.txt", "w") as text_file:
        # Looping over rows to convert each row to libffm format
        datastring = ""
        for r in range(nrows):
            datarow = df.iloc[r].to_dict()
            datastring += str(int(datarow['c_cnt']))
            field_position = 0
            feature_count_index = 0
            for i in range(1, ncols):
                if field_position == feature_count[feature_count_index]:
                    feature_count_index += 1
                    field_position = 0
                # For numerical fields, we are creating a dummy field here
                if (i - 1) < len(numerics):
                    #we are dealing with a numeric feature
                    feat = numerics[i - 1]
                    datastring = datastring + " "+str(i)+":"+ str(i)+":"+ str(datarow[feat])
                else:
                    #categorical feature
                    if datarow[i] == 1:
                        datastring = datastring + " "+str(i)+":"+ str(field_position)+":1"
                    field_position += 1
            datastring += '\n'
        text_file.write(datastring)

In [72]:
# Generate list of numeric feature names
numerics = train_ffm_transformed.columns[1:7].tolist()

In [73]:
# Rename columns to make it easier to transform to libffm format
train_ffm_transformed.columns = ["c_cnt"] + numerics + list(range(7, 142))
test_ffm_transformed.columns = ["c_cnt"] + numerics + list(range(7, 142))

In [74]:
convert_to_ffm(train_ffm_transformed, "train", numerics)
convert_to_ffm(test_ffm_transformed, "test", numerics)

Train FFM model for 15 epochs, achieved 78.5% accuracy, 0.45 log loss and 0.86 AUC score on test set:

In [75]:
# Training task
ffm_model = xl.create_ffm()  
ffm_model.setTrain("./train_ffm.txt")   # Training data
ffm_model.setValidate("./test_ffm.txt")  # Validation data
ffm_model.setSigmoid()

# param:
#  0. binary classification
#  1. learning rate : 0.2
#  2. regular lambda : 0.002
#  3. Metric for monitoring validation set performance (using accuracy rn)
#  4. Maximum number of Epochs
param = {'task':'binary', 'lr':0.2, 'lambda':0.002,'metric':'acc','epoch':15}

# Train model
ffm_model.fit(param, "./model.out")

Find F1 score of test data:

In [88]:
ffm_model.setTest("./test_ffm.txt")
ffm_model.setSign() #makes outputs either 0 or 1
ffm_model.predict("./model.out", "./output.txt")

In [89]:
y_pred_ffm = pd.read_csv('output.txt', header = None)
y_pred_ffm.head()

,0
0,0
1,0
2,0
3,1
4,0


In [95]:
confusion_ffm = confusion_matrix(test_ffm_transformed['c_cnt'], y_pred_ffm) #00 - TN, 10 - FN, 01 - FP, 11 - TP
print("Confusion Matrix:\n", confusion_ffm)

precision_ffm = confusion_ffm[1][1] / (confusion_ffm[1][1] + confusion_ffm[0][1])
recall_ffm = confusion_ffm[1][1] / (confusion_ffm[1][1] + confusion_ffm[1][0])
print("Precision:", precision_ffm, ", Recall:", recall_ffm)

print("F1 Score:", f1_score(test_ffm_transformed['c_cnt'], y_pred_ffm)) #much better F1 score!

Confusion Matrix:
 [[212  32]
 [ 63 172]]
Precision: 0.843137254902 , Recall: 0.731914893617
F1 Score: 0.783599088838


### Test FFM on original data

In [388]:
combined = pd.concat([train_ffm_transformed, test_ffm_transformed])

In [415]:
ffm_original = pd.DataFrame(filtered_subset['c_cnt'])
create_numerical_features(ffm_original, filtered_subset)

In [420]:
ffm_original.reset_index(inplace=True, drop=True)

In [417]:
# Encoding features of original data
ffm_original_encoded_device = le_device.transform(filtered_subset["ua_device_type"].tolist())
ffm_original_encoded_ad_id = encode_feature("advertiser_id", filtered_subset)
ffm_original_encoded_ad_network_id = encode_feature("ad_network_id", filtered_subset)
ffm_original_encoded_geo_country_code = encode_feature("geo_country_code2", filtered_subset)
ffm_original_encoded_ua_os_name = encode_feature("ua_os_name", filtered_subset)
ffm_original_encoded_site_id = encode_feature("site_id", filtered_subset)

In [422]:
# Combined all encoded features
ffm_original = pd.concat([ffm_original, pd.DataFrame(ffm_original_encoded_device),
                                  pd.DataFrame(ffm_original_encoded_ad_id),
                                  pd.DataFrame(ffm_original_encoded_ad_network_id),
                                  pd.DataFrame(ffm_original_encoded_geo_country_code),
                                  pd.DataFrame(ffm_original_encoded_ua_os_name),
                                  pd.DataFrame(ffm_original_encoded_site_id)], 
                         axis=1)

In [423]:
ffm_original.head()

,c_cnt,i_cnt,r_cnt,campaign_type,ctr_rate,num_ads_requested,hour,0,1,2,...,14,15,16,17,18,19,20,21,22,23
0,0.0,1.0,1,1,0.000100,10,6,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0.0,1.0,1,1,0.000100,10,2,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0.0,1.0,1,1,0.000100,10,2,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0.0,1.0,1,1,0.000393,10,4,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0.0,1.0,1,1,0.000100,10,4,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [425]:
# Rename columns to make it easier to transform to libffm format
ffm_original.columns = ["c_cnt"] + numerics + list(range(7, 142))

In [427]:
convert_to_ffm(combined, "train_combined", numerics)

In [426]:
convert_to_ffm(ffm_original, "test_original", numerics)

KeyboardInterrupt: 

In [ ]:
# Training task
ffm_model = xl.create_ffm()  
ffm_model.setTrain("./train_combined_ffm.txt")   # Training data
ffm_model.setValidate("./test_original_ffm.txt")  # Validation data
ffm_model.setSigmoid()

# param:
#  0. binary classification
#  1. learning rate : 0.2
#  2. regular lambda : 0.002
#  3. Metric for monitoring validation set performance (using accuracy rn)
#  4. Maximum number of Epochs
param = {'task':'binary', 'lr':0.2, 'lambda':0.002,'metric':'acc','epoch':5}

# Train model
ffm_model.fit(param, "./model.out")